In [1]:
import numpy as np
import yaml
import pandas as pd
import json
from tqdm import tqdm

In [2]:
SLA = 250
key_name = "rt_ti_u_bound_c90_p95" #rt_ti_u_bound_c90_p95 responseTimes95th
BASE_DIR = '/home/vahid/Dropbox/data/swarm-manager-data/results/nodejs_bookstore/'

In [4]:
workloads = []
for ap in [0.1,0.3,0.5,0.7,0.9]:
    for vus in [400,475,550,625,700]:
        workloads.append(str(vus)+"_120_"+str(ap)+"_10")      
#"300_120_0.3_10",
#"400_120_0.5_10"
# workloads = ["400_120_0.5_10", "500_120_0.65_10", "600_120_0.3_10","700_120_0.5_10"]

In [5]:
workloads

['400_120_0.1_10',
 '475_120_0.1_10',
 '550_120_0.1_10',
 '625_120_0.1_10',
 '700_120_0.1_10',
 '400_120_0.3_10',
 '475_120_0.3_10',
 '550_120_0.3_10',
 '625_120_0.3_10',
 '700_120_0.3_10',
 '400_120_0.5_10',
 '475_120_0.5_10',
 '550_120_0.5_10',
 '625_120_0.5_10',
 '700_120_0.5_10',
 '400_120_0.7_10',
 '475_120_0.7_10',
 '550_120_0.7_10',
 '625_120_0.7_10',
 '700_120_0.7_10',
 '400_120_0.9_10',
 '475_120_0.9_10',
 '550_120_0.9_10',
 '625_120_0.9_10',
 '700_120_0.9_10']

In [9]:
# cpu_util_rules = sorted(['cpu90>50','cpu90>60','cpu90>70','cpu90>80','cpu90>90',
#                          'cpu95>50','cpu95>60','cpu95>70','cpu95>80','cpu95>90',
#                          'cpu.mean>50','cpu.mean>60','cpu.mean>70','cpu.mean>80','cpu.mean>90'])
strategies = {
    'adfc-ti':'/AddDifferentFractionalCPUcores/adfc_utilization_ti_95_'+str(SLA)+'.yml',
    'mc-adfc-ti':'/AddDifferentFractionalCPUcores/mc_adfc_utilization_ti_95_'+str(SLA)+'.yml',

#     'afc-ti-0.33': '/AddFractionalCPUcores/afc_0.33_ti_95_' + str(SLA) + '.yml',
#     'afc-rt-0.33': '/AddFractionalCPUcores/afc_0.33_rt_95_' + str(SLA) + '.yml',
}


In [18]:
def does_specs_meet_sla(config):
    return config['auth']['responseTimes']['total'][key_name] < SLA and \
            config['books']['responseTimes']['total'][key_name] < SLA

temp_dict = {'workloads':[]}
for strategy, file_path in strategies.items():
    temp_dict[strategy] = []
    for workload in workloads:
        if len(workloads) != len(temp_dict['workloads']): temp_dict['workloads'].append(workload)
        
        meetsSLA = False
        cpu_count = 0
        requiredSteps = 0
        with open(BASE_DIR + workload + file_path) as f:
            data = yaml.load(f, Loader=yaml.FullLoader)
        for _step in data['steps']:
            requiredSteps += 1
            cpu_count = _step['specs']['auth']['replicaCount'] * _step['specs']['auth']['CPULimits'] + _step['specs']['books']['replicaCount'] * _step['specs']['books']['CPULimits'] + _step['specs']['gateway']['replicaCount'] * _step['specs']['gateway']['CPULimits']
            if does_specs_meet_sla(_step['info']):
                meetsSLA = True
                break
        
        if not meetsSLA: 
            requiredSteps = 0
            cpu_count = 0
            temp_dict[strategy].append('-')
        else:
#             temp_dict[strategy].append((np.round(cpu_count,2), requiredSteps))
            temp_dict[strategy].append(np.round(cpu_count,2))
#             temp_dict[strategy].append(requiredSteps)

for col, values in temp_dict.items():
    print(col, len(values))
        

workloads 25
adfc-ti 25
mc-adfc-ti 25


In [19]:
df = pd.DataFrame(temp_dict)
df.head(25)

,workloads,adfc-ti,mc-adfc-ti
0,400_120_0.1_10,6.93,7.00
1,475_120_0.1_10,7.86,8.00
2,550_120_0.1_10,8.93,9.93
3,625_120_0.1_10,10.86,11.86
4,700_120_0.1_10,13.72,11.93
5,400_120_0.3_10,7.04,7.04
6,475_120_0.3_10,7.82,7.82
7,550_120_0.3_10,9.06,9.06
8,625_120_0.3_10,10.07,10.07
9,700_120_0.3_10,11.63,12.64


In [20]:
df.mean(axis=0)

adfc-ti       8.8824
mc-adfc-ti    8.9920
dtype: float64